Acá voy a preprocesar el texto con los terminos del G20. El texto tiene frases separadas por un salto de linea.
Las colecciones de frases estan divididas por topicos, estos topicos estan separados por una linea vacia (o mas bien dos saltos de linea)

La idea es:
+ Crear una lista de listas, donde en cada lista habra una coleccion de terminos
+ Por cada coleccion de frases leer un elemento y agregarlo a una lista
    + Si el elemento leído es un espacio en blanco hay que cambiar la lista
    + Si el elemento es "*******************************", finalizar

In [1]:
#Dado un lenguaje, devuelvo un texto conteniendo una lista de terminos a procesar
def abrirArchivoTerms(lang):
    #Defino la carpeta donde se almacenan los G20 TERMS
    path_terms = 'terms/'

    #Abro el archivo donde se encontraran las listas de los terminos
    filepath = path_terms+'G20 TERMS '+lang+'.txt'

    file = open(filepath, 'r', encoding='utf-8')
    text = file.readlines()
    file.close()
    
    return text

In [2]:
#Dada una lista de terminos, devuelvo una lista donde cada elemento es una lista de terminos interrelacionados
def crearListasTerminos(text):
    listas = list()
    sub_list = list()

    for line in text:
        if line != '\n':
            sub_list.append(line)
        else:
            listas.append(sub_list)
            sub_list = list()
    return listas

La lista contiene muchos '\n' y signos de puntuacion. Acá lo proceso antes de agregarlo a la lista

In [3]:
#Dada una lista de listas de string, devuelvo una lista de listas de strings, donde todos sus elementos estan en lowercase
def to_lowercase(listas):
    sentences_lowercase = []
    for lista in listas:
        sub_list = []
        for s in lista:
            sub_list.append(s.lower())
        sentences_lowercase.append(sub_list)
        
    return sentences_lowercase

In [4]:
#Defino funciones para eliminar los elementos innecesarios de los textos
import regex as re

#Funcion para filtrar los hashtags
def remove_hashtags(sentences):
    new_sentences = []
    for s in sentences:
        new_sentence = re.sub(r' #(\w)+','', s)
        if new_sentence != '':
            new_sentences.append(new_sentence)
    return new_sentences

#Funcion para filtrar los signos de puntuacion
def remove_punctuation(sentences):
    new_sentences = []
    for s in sentences:
        new_sentence = re.sub(r'$[-+]+', '', s)
        new_sentence = re.sub(r'[-+]+', ' ', new_sentence)
        new_sentence = re.sub(r'[,.;:]+|\+|\(|\)|\[|\]|\"', '', new_sentence)
        if new_sentence != '':
            final_sentence = re.sub(r'(\s*\n\s*)+', '', new_sentence) #Saco los saltos de linea
            if final_sentence != '':
                new_sentences.append(final_sentence)
    return new_sentences

#Funcion que llama a todos los removedores definidos anteriormente
def remove_all(sentences):
    sentences = remove_hashtags(sentences)
    sentences = remove_punctuation(sentences)
    return sentences

#Dada una lista de listas de textos
    #Devuelvo una lista de listas con los textos 
def remove_all_from_lists(sentences_lowercase):
    sentences_clean = list()
    for sentences in sentences_lowercase:
        sentences_clean.append(remove_all(sentences))
    return sentences_clean

In [5]:
#Quito las stopwords
import spacy
#from nltk.corpus import stopwords

#Dada una lista de listas de frases, devuelvo una lista de listas de frases sin las stopwords
def remove_stopwords(sentences, spacy_model):
    #dic_langs = {'es': 'spanish', 'en': 'english', 'pt': 'portuguese'}
    
    #stop_words = set(stopwords.words(dic_langs[lang]))
    stop_words = spacy.load(spacy_model).Defaults.stop_words
    stop_words.add('amp')
    list_list_sentences_wo_stopwords = list()
    
    for i in range(len(sentences)):
        list_sentences_wo_stopwords = list()
        
        for j in range(len(sentences[i])):
            sentence_wo_stopwords = list()
            
            words = sentences[i][j].split()
            for word in words:
                if(word.lower() not in stop_words and (len(word) > 1)):
                    sentence_wo_stopwords.append(word)
            list_sentences_wo_stopwords.append(' '.join(sentence_wo_stopwords))
            
        list_list_sentences_wo_stopwords.append(list_sentences_wo_stopwords)
    return list_list_sentences_wo_stopwords

In [6]:
import spacy
import lemminflect

#Dado un DataFrame con una lista de frases y un modelo de spacy, paso palabras a sus lemmas
def transform_lemmas(list_list_sentences, spacy_model):
    nlp = spacy.load(spacy_model)

    for i in range(len(list_list_sentences)):
        for j in range(len(list_list_sentences[i])):
            words_lemmas = list()
            words_nlp = nlp(list_list_sentences[i][j])
            for word in words_nlp:
                words_lemmas.append(word._.lemma())
            list_list_sentences[i][j] = ' '.join(words_lemmas)
    return list_list_sentences

Ya que tengo la lista de listas de terminos, puedo ahora armar las consultas.
___
       
**Para forjar una consulta:**
+ Tomar un texto preparado para hacer la consulta (las partes de "query", "bool" no van a cambiar)
+ Crear una funcion que tome una lista de frases
+ Por cada frase F de la lista, agregar las lineas:
    + {"match": {"retweeted_status.extended_tweet.full_text": {"query": "F"}}}
    + {"match_phrase": {"text": {"query": "F"}}}
+ (si no es la ultima frase en la lista, agregar una coma al final)

---
20/05/2021:

Cambiaron un toquecin los requerimientos. Ahora agrego en la consulta:
+ El periodo de tweets que quiero obtener en base al G20 de cada año
    - Armo un diccionario que tiene las fechas de inicio y fin
+ Las menciones a distintos usuarios por pais
    - Armo un diccionario con los @ de los distintos lideres en base a los distintos años

Variables de configuracion para elegir el año y el/los lideres mencionados

In [7]:
#Funcion que recibe un iterador con nombres de cuentas de tweets y devuelve un string con esa lista de menciones
def generar_user_mentions(iterador_users):
    filter_terms_users_ini = '\t{"terms": {"entities.user_mentions.screen_name": ['
    filter_terms_users_fin = ' }},'
    toReturn = filter_terms_users_ini + '"'
    seguir = True
    user = next(iterador_users,0)
    if user!=0:
        toReturn += user
        toReturn += '"'
    while(seguir):
        user = next(iterador_users,0)
        if user!=0:
            toReturn += ',"'
            toReturn += user
            toReturn += '"'
        else:
            seguir = False
    toReturn += ']'
    toReturn += filter_terms_users_fin
    return toReturn

In [8]:
#Funcion que recibe un iterador con frases, un string de inicio y un string de fin
def generar_matches(iterador, ini, fin):
    matches = ''
    
    seguir = True
    frase = next(iterador, 0)
    while (seguir):
        if frase != 0: #Si no es la ultima frase, agrego un objeto 'query' con la frase. Si es la ultima, termino el loop
            matches += ini + frase + fin
        else:
            seguir = False

        frase = next(iterador, 0)
        if frase != 0: #Si esta no es la ultima frase agrego una coma a la consulta
            matches += ',\n'
    matches += '\n'
    return matches


In [9]:
#Funcion que recibe un entero, recibe unos rangos y devuelve un string con el filtro del rango de fechas a consultar
def generar_range_dates(ranges):
    filter_range_dates_ini = '''
    \t{"range": {
          "created_at": {
            "gte": "'''
    filter_range_dates_mid = '''",
            "lte": "'''
    filter_range_dates_fin = '''"
          }
        }},'''
    toReturn = filter_range_dates_ini + ranges[0] + filter_range_dates_mid + ranges[1] + filter_range_dates_fin
    
    return toReturn

## Automatizo el proceso de procesar las consultas y guardar los resultados en los archivos

In [10]:
def generarConsulta(anio, lista_usuarios, lista_claves):
    ##############################
    ####DEFINICION DE CONSTANTES##
    ##############################
    
    #Partes de la consulta
    #La forma de la consulta sería: 
    #<consulta_ini> [<match_rt_ini> <frase> <match_rt_fin>]+ <consulta_med> [<match_t_ini> <frase> <match_t_fin>]+ <consulta_fin>

    #Esto es el inicio de la consulta. Llega hasta la lista de matches con 'retweeted_status'
    consulta_ini = '''{
      "_source": ["created_at", "text", "extended_tweet.full_text", "retweeted_status.extended_tweet.full_text"],
      "query": {
        "bool": {
          "minimum_should_match": 1,
          "should": [
    '''

    #Armo el esqueleto de la consulta para el campo text
    match_t_ini = '\t\t{"match": {"text": {"query": "'
    match_t_fin = '''", 
                        "auto_generate_synonyms_phrase_query" : true, 
                        "fuzzy_transpositions": true,
                        "fuzziness": "2" }}}'''

    #Armo el esqueleto de la consulta para el campo extended_tweet.full_text
    match_et_ini = '\t\t{"match": {"extended_tweet.full_text": {"query": "'
    match_et_fin = '''", 
                        "auto_generate_synonyms_phrase_query" : true, 
                        "fuzzy_transpositions": true,
                        "fuzziness": "1" }}}'''

    #Armo el esqueleto de la consulta para el campo retweeted_status.extended_tweet.full_text
    match_rt_ini = '\t\t{"match": {"retweeted_status.extended_tweet.full_text": {"query": "'
    match_rt_fin = '''", 
                        "auto_generate_synonyms_phrase_query" : true, 
                        "fuzzy_transpositions": true,
                        "fuzziness": "1" }}}'''

    consulta_med = ''',
    '''

    consulta_fin_1 = '''
            ],
          "filter": [
    '''

    #Por ahora este es un filtro sin usar
    filter_terms_hashtags = '{"terms": {"entities.hashtags.text": '+ hashtags +'}},'


    consulta_fin_2 = '''
            {"term":{"lang": "'''+lang+'''"}}
          ]
        }
      }
    }
    '''
    
    #############################
    #CONSTRUCCION DE LA CONSULTA#
    #############################
    
    #Armo la consulta, iterando a traves de una de las listas
    consulta = consulta_ini

    #Genero los "match" para el campo "text"
    iter_list = iter(lista_claves)
    consulta += generar_matches(iter_list,match_t_ini,match_t_fin) + ','

    #Genero los "match" para el campo "extended_tweet.full_text"
    iter_list = iter(lista_claves)
    consulta += generar_matches(iter_list,match_et_ini,match_et_fin) + ','
    
    #Genero los "match" para el campo "retweeted_status.extended_tweet.full_text"
    iter_list = iter(lista_claves)
    consulta += generar_matches(iter_list,match_rt_ini,match_rt_fin)
    
    consulta += consulta_fin_1

    #Agrego la linea de terms al contexto de filtro
    iter_user_mentions = iter(lista_usuarios)
    consulta += generar_user_mentions(iter_user_mentions)

    consulta += generar_range_dates(g20_ranges)

    consulta += consulta_fin_2
    
    return consulta

In [11]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers

es = Elasticsearch(timeout=2400)
#Dado el cuerpo de una query, realizo una consulta a ES
def consultarES(query):
    return helpers.scan(es, query = body, index = 'tweets', scroll = '2m', preserve_order=True, request_timeout=2000)

In [12]:
#Dado un texto y una lista de textos, busco si el texto pasado por parametro se encuentra en la lista de textos
#Devuelvo un boolean que indica si el texto fue encontrado o no
def exists(text, list_texts):
    exist = False
    
    for t in list_texts:
        if t ==text:
            exist = True
            break
            
    return exist

In [13]:
#Dados los resultados de una consulta en ES y la cantidad de tweets que quiero obtener
 #Tomo los campos created_at, _score y los textos completos de los tweets
 #Y devuelvo un diccionario con estos campos
 #para despues crear un dataframe y guardarlo en un csv
def obtenerFrases(res,cant_tweets):
    dic_resultados = {'created_at':[], 'text':[], '_score':[]}

    for i in range(cant_tweets):
        try:
            hit = res.__next__()
            if hit['_source'].get('extended_tweet') != None:
                text = hit['_source']['extended_tweet'].get('full_text')
            else:
                if hit['_source'].get('retweeted_status') != None and hit['_source'].get('retweeted_status').get('extended_tweet') != None:
                    text = hit['_source']['retweeted_status']['extended_tweet'].get('full_text')
                else:
                    text = hit['_source']['text']
            #Solo agrego tweets que no hayan sido escritos antes
            if text != '' and not exists(text, dic_resultados['text']):
                dic_resultados['created_at'].append(hit['_source'].get('created_at'))
                dic_resultados['_score'].append(hit.get('_score'))
                text = text.replace('\t',' ') #Planeo usar las tabulaciones como separador de celdas
                text = text.replace('\n',' ')#Es para evitar saltos de linea en un mismo tweet
                dic_resultados['text'].append(text)
        except StopIteration:
            pass
    
    return dic_resultados

In [14]:
import pandas as pd

#Dado un diccionario, lo paso a un Dataframe de Pandas y lo guardo en un archivo
def crearArchivo(anio, pais, tema, dic_resultados):
    #Guardo los resultados de las consultas
    filename = 'results/' + str(anio) + '/' + lang + '/' + str(pais) + " " + str(anio) + ' - Tema ' + str(tema) + '.csv'
    print(filename)
    df = pd.DataFrame(dic_resultados)
    df.to_csv(filename, encoding='utf-8', index=False, sep='\t')

In [15]:
def crearListaUsuarios(anio, pais):
    lista_usuarios = list()
    
    index_tupla_usuarios = 2018 - anio
    #Creo una lista de cuentas de usuario en lowercase, ya que voy a aplicar terms en el filtro de usuarios
     #ya que "terms" no aplica un analyzer y (aparentemente) se guardan los usuarios en ES en minusculas, necesito hacer este paso
    for elem in tuple_dics_lists_liders[index_tupla_usuarios][pais]:
        lista_usuarios.append(elem.lower())
        
    return lista_usuarios

In [16]:
tuple_dics_lists_liders = (
    #2018
    {'alemania': {'AngelaMerkeICDU'}, 'arabia': {'Saudiegov'},
    'argentina': ('mauriciomacri', 'CasaRosada'), 'australia': {'ScottMorrisonMP', 'thepmo'},
    'brasil': {'BrazilGovNews', 'MichelTemer'}, 'canada': {'JustinTrudeau', 'CanadianPM'},
    'china': {'chinascio'}, 'corea del sur': {'TheBlueHouseENG', 'moonriver365'}, 
    'usa': ('POTUS','USAgov','WhiteHouse','realdonaldtrump'), 'francia': {'gouvernementFR','French_Gov', 'EmmanuelMacron', 'Elysee'},
    'india': {'rashtrapatibhvn','PMOIndia','narendramodi'}, 'indonesia': {'jokowi'},
    'italia': {'Quirinale', 'GiuseppeConteIT'}, 'japon': {'JapanGov', 'AbeShinzo', 'JPN_PMO'},
    'mexico': {'PresidenciaMX', 'EPN'}, 'uk': {'cabinetofficeuk', '10DowningStreet', 'theresa_may'},
    'rusia': {'PutinRF_Eng','GovernmentRF','KremlinRussia_E','MedvedevRussia'}, 'sudafrica': {'PresidencyZA', 'CyrilRamaphosa', 'GovernmentZA'},
    'turquia': {'RT_Erdogan', 'Communications', 'trpresidency'}, 'eu': {'eucopresident','EU_Commission'}},
    #No tenemos datos para el 2019
    #2020
    {'alemania': {'AngelaMerkeICDU'}, 'arabia': {'Saudiegov'},
    'argentina': ('alferdez', 'CasaRosada'), 'australia': {'ScottMorrisonMP', 'thepmo'},
    'brasil': {'BrazilGovNews', 'jairbolsonaro'}, 'canada': {'JustinTrudeau', 'CanadianPM'},
    'china': {'chinascio'}, 'corea del sur': {'TheBlueHouseENG', 'moonriver365'}, 
    'usa': ('POTUS','USAgov','WhiteHouse','realdonaldtrump'), 'francia': {'EmmanuelMacron', 'Elysee', 'gouvernementFR', 'French_Gov'},
    'india': {'rashtrapatibhvn', 'PMOIndia', 'narendramodi'}, 'indonesia': {'jokowi'},
    'italia': {'Quirinale', 'GiuseppeConteIT'}, 'japon': {'JapanGov', 'AbeShinzo', 'JPN_PMO'},
    'mexico': {'PresidenciaMX', 'lopezobrador_'}, 'uk': {'cabinetofficeuk','10DowningStreet', 'BorisJohnson'},
    'rusia': {'PutinRF_Eng', 'GovernmentRF', 'KremlinRussia_E', 'MedvedevRussia'}, 'sudafrica': {'PresidencyZA', 'CyrilRamaphosa', 'GovernmentZA'},
    'turquia': {'RT_Erdogan', 'Communications', 'trpresidency'}, 'eu': {'eucopresident', 'EU_Commission'}},
    #2021
    {'alemania': {'AngelaMerkeICDU'}, 'arabia': {'Saudiegov'},
    'argentina': ('alferdez', 'CasaRosada'), 'australia': {'ScottMorrisonMP', 'thepmo'},
    'brasil': {'BrazilGovNews', 'jairbolsonaro'}, 'canada': {'JustinTrudeau', 'CanadianPM'},
    'china': {'chinascio'}, 'corea del sur': {'TheBlueHouseENG'}, 
    'usa': ('POTUS','USAgov','WhiteHouse','JoeBiden'), 'francia': {'EmmanuelMacron', 'Elysee', 'gouvernementFR', 'French_Gov'},
    'india': {'rashtrapatibhvn', 'PMOIndia', 'narendramodi'}, 'indonesia': {'jokowi'},
    'italia': {'Quirinale'}, 'japon': {'JapanGov', 'sugawitter', 'JPN_PMO'},
    'mexico': {'PresidenciaMX', 'lopezobrador_'}, 'uk': {'cabinetofficeuk', '10DowningStreet', 'BorisJohnson'},
    'rusia': {'PutinRF_Eng', 'GovernmentRF', 'KremlinRussia_E', 'MedvedevRussia'}, 'sudafrica': {'PresidencyZA', 'CyrilRamaphosa', 'GovernmentZA'},
    'turquia': {'RT_Erdogan', 'Communications', 'trpresidency'}, 'eu': {'eucopresident', 'EU_Commission'}},)

In [17]:
#Hago una lista de listas de dos elementos
 #En cada sublista pongo el inicio de la fecha de cada año del g20 y la fecha de fin
dic_year_ranges = {
    2018: ["Thu Nov 01 00:00:00 +0000 2018", "Mon Dec 31 23:59:59 +0000 2018"],
    2020: ["Sat Feb 01 00:00:00 +0000 2020", "Thu Apr 30 23:59:59 +0000 2020"],
    2021: ["Fri Jan 01 00:00:00 +0000 2021", "Fri Dec 31 23:59:59 +0000 2021"]
}

In [18]:
#Defino la cantidad de tweets, las cuales quiero obtener (a traves de helpers.scan())
cant_tweets = 50000
#Defino la lista de paises a buscar, mediante el idioma usado
dic_lang_countries = {'en':['alemania', 'arabia', 'australia', 'brasil', 'canada', 'china', 'corea del sur', 'francia', 
                            'india', 'indonesia', 'italia', 'japon', 'uk', 'rusia', 'sudafrica', 'turquia', 'eu', 'usa'],
                      'es':['argentina', 'mexico'],
                      'pt':['brasil']}
dic_lang_countries = {'en':['brasil']}
#Defino la lista de hashtags que representa el summit de cada año
dic_hashtags = {2018:'["g20","g20summit","g20argentina"]',
                2020:'["g20","g20summit","g20saudiarabia"]',
                2021:'["g20","g20summit","g20italy"]'}
#Defino los modelos de Spacy a usar segun el lenguaje
dic_spacy_models = {'en': 'en_core_web_lg', 'es': 'es_core_news_lg', 'pt': 'pt_core_news_lg'}

In [21]:
import json

%load_ext jupyternotify
langs = ['en', 'es', 'pt']
years = [2018,2020,2021]

for lang in langs:
    #Formo las listas de consultas
    text = abrirArchivoTerms(lang)
    listas = crearListasTerminos(text)
    sentences_lowercase = to_lowercase(listas)
    listas_temas = remove_all_from_lists(sentences_lowercase)
    listas_temas = remove_stopwords(listas_temas, dic_spacy_models[lang])
    listas_temas = transform_lemmas(listas_temas, dic_spacy_models[lang])
    countries_list = dic_lang_countries[lang]
    for year_g20 in years:
        g20_ranges = dic_year_ranges[year_g20]
        hashtags = dic_hashtags[year_g20]
        file_querys = open('consultas'+str(year_g20)+'.txt', 'a', encoding='utf-8')
        
        for country in countries_list:
            for subject in range(len(listas_temas)):
                lista_usuarios = crearListaUsuarios(year_g20, country)
                lista_querys = generarConsulta(year_g20, lista_usuarios, listas_temas[subject])
                body = json.loads(lista_querys)
                resultado = consultarES(body)
                dic_resultados = obtenerFrases(resultado, cant_tweets)
                crearArchivo(year_g20, country, subject, dic_resultados)
                
                print("Finalizo " + str(country) + "-" + str(year_g20) + ' Tema: ' + str(subject))
                file_querys.write(lista_querys)
                file_querys.write('\n\n********************************\n\n')
file_querys.close()

%notify

<IPython.core.display.Javascript object>

results/2018/en/brasil 2018 - Tema 0.csv
Finalizo brasil-2018 Tema: 0
results/2018/en/brasil 2018 - Tema 1.csv
Finalizo brasil-2018 Tema: 1
results/2018/en/brasil 2018 - Tema 2.csv
Finalizo brasil-2018 Tema: 2
results/2018/en/brasil 2018 - Tema 3.csv
Finalizo brasil-2018 Tema: 3
results/2020/en/brasil 2020 - Tema 0.csv
Finalizo brasil-2020 Tema: 0
results/2020/en/brasil 2020 - Tema 1.csv
Finalizo brasil-2020 Tema: 1
results/2020/en/brasil 2020 - Tema 2.csv
Finalizo brasil-2020 Tema: 2
results/2020/en/brasil 2020 - Tema 3.csv
Finalizo brasil-2020 Tema: 3
results/2021/en/brasil 2021 - Tema 0.csv
Finalizo brasil-2021 Tema: 0
results/2021/en/brasil 2021 - Tema 1.csv
Finalizo brasil-2021 Tema: 1
results/2021/en/brasil 2021 - Tema 2.csv
Finalizo brasil-2021 Tema: 2
results/2021/en/brasil 2021 - Tema 3.csv
Finalizo brasil-2021 Tema: 3


<IPython.core.display.Javascript object>

In [23]:
#TEMP
import json

%load_ext jupyternotify
lang = 'en'#,'es','pt']
years = [2021]#,2020,2021]

#Formo las listas de consultas
text = abrirArchivoTerms(lang)
listas = crearListasTerminos(text)
sentences_lowercase = to_lowercase(listas)
listas_temas = remove_all_from_lists(sentences_lowercase)
listas_temas = remove_stopwords(listas_temas, dic_spacy_models[lang])
listas_temas = transform_lemmas(listas_temas, dic_spacy_models[lang])
countries_list = dic_lang_countries[lang]
for year_g20 in years:
    g20_ranges = dic_year_ranges[year_g20]
    hashtags = dic_hashtags[year_g20]    
    for country in ['usa']:
        for subject in range(len(listas_temas)):
            lista_usuarios = crearListaUsuarios(year_g20, country)
            lista_querys = generarConsulta(year_g20, lista_usuarios, listas_temas[subject])
            body = json.loads(lista_querys)
            print(body)
            print('\n\n\n')
            #resultado = consultarES(body)
            #dic_resultados = obtenerFrases(resultado, cant_tweets)
            #crearArchivo(year_g20, country, subject, dic_resultados)
            
            print("Finalizo " + str(country) + "-" + str(year_g20) + ' Tema: ' + str(subject))
%notify

The jupyternotify extension is already loaded. To reload it, use:
  %reload_ext jupyternotify
{'_source': ['created_at', 'text', 'extended_tweet.full_text', 'retweeted_status.extended_tweet.full_text'], 'query': {'bool': {'minimum_should_match': 1, 'should': [{'match': {'text': {'query': 'global economic recovery', 'auto_generate_synonyms_phrase_query': True, 'fuzzy_transpositions': True, 'fuzziness': '2'}}}, {'match': {'text': {'query': 'exchange rate commitment', 'auto_generate_synonyms_phrase_query': True, 'fuzzy_transpositions': True, 'fuzziness': '2'}}}, {'match': {'text': {'query': 'trade investment', 'auto_generate_synonyms_phrase_query': True, 'fuzzy_transpositions': True, 'fuzziness': '2'}}}, {'match': {'text': {'query': 'multilateral trading system', 'auto_generate_synonyms_phrase_query': True, 'fuzzy_transpositions': True, 'fuzziness': '2'}}}, {'match': {'text': {'query': 'free trade', 'auto_generate_synonyms_phrase_query': True, 'fuzzy_transpositions': True, 'fuzziness': '2

<IPython.core.display.Javascript object>